In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
import numpy as np
from data_work import data_split
from data_work import processed
from data_work import analyse_dataset
from data_work import set_data
from sklearn.model_selection import train_test_split
import pandas as pd
data=pd.read_csv("/Users/mac/Dev/2022_SE03/data/lasted_data.csv",index_col=None)
data=data.drop("Unnamed: 0",axis=1)

In [2]:

traindata,test_data=train_test_split(data,test_size=0.4,random_state=42,stratify=data['subcategory '])
data = traindata.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)
client_data,server_data=train_test_split(traindata,test_size=0.001,random_state=42)
client_data = client_data.reset_index(drop=True)
server_data = server_data.reset_index(drop=True)

In [3]:

testdata_list=set_data([test_data])[0]

#chia data thành list gồm 2 phàn tử [0] là X, [1] là y

len X 2811, len y 2811


# iid dataset setup 

In [4]:
from model import Net

##Create model function

In [5]:
import torch
import torch.nn as nn

In [6]:
trainloaders,testloaders=data_split(client_data,"subcategory ",8,0.1,IID=1)

prepairing IID dataset
len of client data :4210, len of each subdata : 526
client number 1 : train(474), val(52)
client number 2 : train(474), val(52)
client number 3 : train(474), val(52)
client number 4 : train(474), val(52)
client number 5 : train(474), val(52)
client number 6 : train(474), val(52)
client number 7 : train(474), val(52)
client number 8 : train(474), val(52)


In [7]:
names=["subcategory "]
a=analyse_dataset(trainloaders[7],names)
print(a)

thống kê nhãn subcategory :
3 62
1 67
2 62
6 62
7 68
4 75
5 69
0 9
[3, 1, 2, 6, 7, 4, 5, 0]


##Client

In [8]:
class client():
    def __init__(self,cid,net):
        self.cid=cid 
        self.model=net
        self.trainset=trainloaders[cid]
        self.client_label=analyse_dataset(trainloaders[cid],["subcategory "])
        self.discrete_columns=list(trainloaders[cid].columns)

    def fit(self ):
        # Read values from config
        self.model.fit(self.trainset, self.discrete_columns)
        return "ok"
    
    def Gen_fake_data(self,n_samples):
        synthetic_data = self.model.sample(n_samples)
        return synthetic_data



In [9]:
config = {
    "server_round": 3,
    "local_epochs": 5
    
}

In [10]:
from ctgan import CTGAN
from ctgan import load_demo

def client_fn(cid: str,config): 
    server_round = config["server_round"]
    local_epochs = config["local_epochs"]
    net = CTGAN(epochs=local_epochs)
    print(cid)
    return client(cid, net)

In [11]:
te=torch.tensor(server_data.values)
X_te=te[:,1:]
y_te=te[:,:1]
print(len(X_te))
X_te=X_te.float()
y_te=y_te.squeeze().tolist()
y_te=torch.tensor(y_te)
mul=Net(0.00)
mul.fit(X_te,y_te,0.003,0.2,10)
_,accuracy=mul.evaluate(testdata_list[0],testdata_list[1])
print(f"total accuracy : {accuracy}")

5
accuracy of epoch 0 is 0.0, loss : 108.02294921875
accuracy of epoch 1 is 1.0, loss : 9.291299819946289
accuracy of epoch 2 is 0.0, loss : 1986.9154052734375
accuracy of epoch 3 is 1.0, loss : 108.58467102050781
accuracy of epoch 4 is 1.0, loss : 19.584693908691406
accuracy of epoch 5 is 1.0, loss : 0.0
accuracy of epoch 6 is 1.0, loss : 0.0
accuracy of epoch 7 is 1.0, loss : 0.0
accuracy of epoch 8 is 1.0, loss : 0.0
accuracy of epoch 9 is 1.0, loss : 0.0
total accuracy : 0.3208822483102099


##Server

In [12]:
import random
class server():
    def __init__(self):
        self.server_model=Net(0.00)
        self.trainset=server_data
        self.testset=testdata_list
        self.clients=[]
        n_clients=8
        config = {
            "server_round": round,
            "local_epochs": 5
        }
        for i in range(n_clients):
            client=client_fn(i,config)
            self.clients.append(client)

    
    #def fit_round():
    def simulation(self,rounds):
        
        for round in range(rounds):
            total_fake=[]
            for i in range(len(self.clients)):
                #kết hợp data sever và xáo trộn
                data = pd.concat([self.clients[i].trainset, self.trainset], axis=0)
                print(f"training client {i} with {len(data)} samples")
                data=data.sample(frac=1).reset_index(drop=True)
                self.clients[i].trainset= data
                #D_parameters,G_parameters=client.get_parameter()
                results=self.clients[i].fit()
                data_fake=self.clients[i].Gen_fake_data((round+1)*5)
                #merge new synthetic data to trainset server
                total_fake.append(data_fake)
            tensor_fake=[]
            for data_fake in total_fake:
                self.trainset=pd.concat([data_fake, self.trainset], axis=0)
                self.trainset=self.trainset.sample(frac=1).reset_index(drop=True)
                tensor_fake.append(torch.tensor(data_fake.values))
    
            #get round data to train servermodel
            round_data = torch.empty(1, 29)
            for fake in tensor_fake:
                fake=fake.detach()
                round_data=torch.cat((round_data,fake), dim=0)
            print(f"number of synthetic data : {len(round_data)}")

            #train server model
            X=round_data[:,1:]
            y=round_data[:,:1]
            y=y.squeeze().tolist()
            y=torch.tensor(y)
            #self.server_model.fit(X,y,0.003,0.2,15)
            self.server_model.Cross_validation(X,y,5,0.003)

            #evaluate round accuracy
            _,accuracy=self.server_model.evaluate(self.testset[0],self.testset[1])
            print(f"round{round} accuracy:{accuracy}")


            

            


In [13]:
fed=server()

0
thống kê nhãn subcategory :
6 69
7 62
4 74
3 63
5 59
1 77
2 60
0 10
1
thống kê nhãn subcategory :
7 67
6 72
2 59
3 79
1 64
5 60
4 64
0 9
2
thống kê nhãn subcategory :
3 64
2 64
5 74
1 67
4 67
0 7
6 71
7 60
3
thống kê nhãn subcategory :
3 64
7 64
5 75
1 60
2 70
4 69
6 68
0 4
4
thống kê nhãn subcategory :
7 77
5 63
6 63
3 58
1 72
2 77
4 52
0 12
5
thống kê nhãn subcategory :
3 69
1 70
6 63
7 68
2 76
5 74
4 52
0 2
6
thống kê nhãn subcategory :
7 80
5 66
2 66
6 58
0 11
4 80
3 56
1 57
7
thống kê nhãn subcategory :
3 62
1 67
2 62
6 62
7 68
4 75
5 69
0 9


In [14]:
fed.simulation(5)

training client 0 with 479 samples
training client 1 with 479 samples
training client 2 with 479 samples
training client 3 with 479 samples
training client 4 with 479 samples
training client 5 with 479 samples
training client 6 with 479 samples
training client 7 with 479 samples
number of synthetic data : 41
length of trainset 32, length of testset9
Accuracy for fold 1 : 0.1111111111111111
length of trainset 33, length of testset8
Accuracy for fold 2 : 0.875
length of trainset 33, length of testset8
Accuracy for fold 3 : 1.0
length of trainset 33, length of testset8
Accuracy for fold 4 : 1.0
length of trainset 33, length of testset8
Accuracy for fold 5 : 1.0
round0 accuracy:0.18783351120597652
training client 0 with 524 samples
training client 1 with 524 samples
training client 2 with 524 samples
training client 3 with 524 samples
training client 4 with 524 samples
training client 5 with 524 samples
training client 6 with 524 samples
training client 7 with 524 samples
number of synthet